In [ ]:
from MCS_similarity import *
import pandas as pd

df = pd.read_csv('USPTO_mutations.csv')
all_smiles = df['molecule'].tolist()
all_mutations = df['mutations'].tolist()

In [ ]:
idxs = np.random.choice(df.index.tolist(), 200)
smiles_list = [all_smiles[i] for i in idxs]
len(smiles_list)

In [ ]:
# finding the most similar molecule in dataset
import math
from tqdm import tqdm

Tan_Sims = {}
rdkit_MCS_Sims = {}
fast_MCS_Sims = {}
hybrid_Sims = {}

for i, j in enumerate(idxs):
    smi1 = all_smiles[i]
    mol1 = Chem.MolFromSmiles(smi1)
    print (i, j, smi1)
    recorder1 = sim_recorder(smi1)
    recorder2 = sim_recorder(smi1)
    recorder3 = sim_recorder(smi1)
    recorder4 = sim_recorder(smi1)
    mutations = eval(all_mutations[i])
    for smi2 in mutations:
        mol2 = Chem.MolFromSmiles(smi2)
        t1, sim1 = Tanimoto_Sim(mol1, mol2)
        t2, sim2 = rdkit_MCS_Sim(mol1, mol2)
        t3, sim3 = fast_MCS_Sim(mol1, mol2)
        t4 = t1+t3
        sim4 = math.sqrt(sim1*sim3) # hybrid similarity
        recorder1.record(t1, sim1)
        recorder2.record(t2, sim2)
        recorder3.record(t3, sim3)
        recorder4.record(t4, sim4)
    Tan_Sims[i] = recorder1.summerize(mutations)
    rdkit_MCS_Sims[i] = recorder2.summerize(mutations)
    fast_MCS_Sims[i] = recorder3.summerize(mutations)
    hybrid_Sims[i] = recorder4.summerize(mutations)

In [ ]:
output_dict = {'Tanimoto': Tan_Sims, 'fast_MCS': fast_MCS_Sims, 'rdkit_MCS':rdkit_MCS_Sims, 'hybrid': hybrid_Sims}
for k, v in output_dict.items():
    print (k)
    df = pd.DataFrame.from_dict(v, orient = 'index')
    df.columns = ['time', 'similar_mols']
    df['smiles'] = [all_smiles[i] for i in v.keys()]
    df.to_csv('outputs/%s_Mutations.csv' % k)